In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stomach_Cancer"
cohort = "GSE147163"

# Input paths
in_trait_dir = "../../input/GEO/Stomach_Cancer"
in_cohort_dir = "../../input/GEO/Stomach_Cancer/GSE147163"

# Output paths
out_data_file = "../../output/preprocess/Stomach_Cancer/GSE147163.csv"
out_gene_data_file = "../../output/preprocess/Stomach_Cancer/gene_data/GSE147163.csv"
out_clinical_data_file = "../../output/preprocess/Stomach_Cancer/clinical_data/GSE147163.csv"
json_path = "../../output/preprocess/Stomach_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE147163_family.soft.gz', 'GSE147163_series_matrix.txt.gz']
Identified SOFT files: ['GSE147163_family.soft.gz']
Identified matrix files: ['GSE147163_series_matrix.txt.gz']

Background Information:
!Series_title	"Molecular subtypes in gastric cancer. [III]"
!Series_summary	"We identified the molecular subtypes and conserved modules in gastric cancer by unsupervised clustering algorithm. We defined six molecular signatrues of gastric cancer associated with the biological heterogeneity of gastric cancer and clinical outcome of patients."
!Series_overall_design	"We obtained gene expression profile of gastrectomy samples from 401 gastric cancer patients by HumanHT-12 v3.0 Expression BeadChip array (Illumina). Total RNA was extracted from the fresh-frozen gastrectomy specimens at the Yonsei University Severance Hospital (South Korea) between 2000 and 2010."

Sample Characteristics Dictionary:
{0: ['tissue: gastric cancer']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# This dataset appears to be a gene expression microarray data (HumanHT-12 v3.0 Expression BeadChip array)
# from the background information, so gene expression data should be available
is_gene_available = True

# 2.1 Data Availability
# From the sample characteristics, we only have 'tissue: gastric cancer'
# This indicates all samples are gastric cancer tissue, without control samples
# There is no explicit trait variable that differentiates between cases and controls
# There is also no age or gender information available in the sample characteristics

trait_row = None  # No trait variable that differentiates between cases and controls
age_row = None    # No age information available
gender_row = None # No gender information available

# 2.2 Data Type Conversion (defining functions even though they won't be used in this case)
def convert_trait(value):
    """Convert trait value to binary (0 for control, 1 for case)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # No conversion rule needed as we don't have trait data
    return None

def convert_age(value):
    """Convert age value to continuous"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # No conversion rule needed as we don't have age data
    return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # No conversion rule needed as we don't have gender data
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction - Skip as trait_row is None
# No clinical feature extraction is needed as we don't have trait data


False

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052',
       'ILMN_1343059', 'ILMN_1343061', 'ILMN_1343062', 'ILMN_1343063',
       'ILMN_1343064', 'ILMN_1343291', 'ILMN_1343295', 'ILMN_1343296',
       'ILMN_1343297', 'ILMN_1343298', 'ILMN_1343299', 'ILMN_1343301',
       'ILMN_1343302', 'ILMN_1343303', 'ILMN_1343304', 'ILMN_1343305'],
      dtype='object', name='ID')

Gene expression data shape: (49576, 50)


### Step 4: Gene Identifier Review

In [5]:
# These identifiers with prefix "ILMN_" are Illumina BeadArray probe IDs, not human gene symbols.
# They need to be mapped to human gene symbols for downstream analysis.
# The "ILMN_" prefix indicates these are from Illumina microarray platforms.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Gene annotation preview:
{'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174', 'ILMN_1796063', 'ILMN_1811966'], 'nuID': ['rp13_p1x6D80lNLk3c', 'NEX0oqCV8.er4HVfU4', 'KyqQynMZxJcruyylEU', 'xXl7eXuF7sbPEp.KFI', '9ckqJrioiaej9_ajeQ'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'Unigene', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'Transcript': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'ILMN_Gene': ['LOC23117', 'HS.575038', 'FCGR2B', 'TRIM44', 'LOC653895'], 'Source_Reference_ID': ['XM_933824.1', 'Hs.575038', 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'RefSeq_ID': ['XM_933824.1', nan, 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'Unigene_ID': [nan, 'Hs.575038', nan, nan, nan], 'Entrez_Gene_ID': [23117.0, nan, 2213.0, 54765.0, 653895.0], 'GI': [89040007.0, 10437021.0, 88952550.0, 29029528.0, 89033487.0], 'Access

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in the gene annotation contain identifiers and symbols
# Looking at the gene annotation preview and gene expression data:
# - The gene expression data index contains probe IDs like 'ILMN_1343048'
# - The gene annotation has 'ID' column with similar values like 'ILMN_1725881'
# - The gene symbols appear to be in the 'Symbol' column

# 2. Get gene mapping dataframe
probe_col = 'ID'  # Column containing probe identifiers
symbol_col = 'Symbol'  # Column containing gene symbols

# Extract mapping data and handle potential issues
mapping_df = get_gene_mapping(gene_annotation, probe_col, symbol_col)

print(f"Created gene mapping with {len(mapping_df)} entries.")
print("First few rows of mapping data:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

print(f"\nConverted to gene expression data with {len(gene_data)} genes and {gene_data.shape[1]} samples.")
print("First few genes:")
print(gene_data.index[:10])

# Preview the first few rows of gene expression data
print("\nPreview of gene expression data:")
print(gene_data.iloc[:5, :5])


Created gene mapping with 36157 entries.
First few rows of mapping data:
             ID       Gene
0  ILMN_1725881   LOC23117
2  ILMN_1804174     FCGR2B
3  ILMN_1796063     TRIM44
4  ILMN_1811966  LOC653895
5  ILMN_1668162    DGAT2L3



Converted to gene expression data with 19120 genes and 50 samples.
First few genes:
Index(['A1BG', 'A1CF', 'A26A1', 'A26B1', 'A26C1B', 'A26C3', 'A2BP1', 'A2M',
       'A2ML1', 'A3GALT2'],
      dtype='object', name='Gene')

Preview of gene expression data:
        GSM4419484  GSM4419485  GSM4419486  GSM4419487  GSM4419488
Gene                                                              
A1BG      9.853618   10.537068    9.865281    9.937021    9.831023
A1CF     23.955097   20.615845   15.110226   14.553892   18.103539
A26A1     9.452464    9.388353    9.605953    9.653673    9.743757
A26B1     4.776131    4.864010    4.798284    4.826584    4.919554
A26C1B    4.971237    4.679716    4.754202    4.698224    4.954362


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Create a minimal DataFrame with a dummy trait column since no clinical data is available
# For datasets where all samples are the same class (all gastric cancer), we need to mark them all the same
linked_data = normalized_gene_data.T.copy()
linked_data[trait] = 1  # All samples marked as cases (gastric cancer)
print(f"Created linked data with dummy trait column. Shape: {linked_data.shape}")

# 3. Handle missing values in gene data
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Since all samples have the same trait value (all are gastric cancer), the dataset is biased by definition
is_trait_biased = True
print(f"Trait distribution assessment: dataset is biased (all samples are gastric cancer)")

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,
    is_trait_available=False,  # Although we created a dummy trait column, the actual trait data isn't available
    is_biased=is_trait_biased, 
    df=linked_data,
    note="Dataset contains gene expression data from gastric cancer samples but lacks control samples or trait differentiation."
)

# 6. No need to save the linked data as it's not usable for trait association
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues (no trait differentiation available).")

Normalized gene data shape: (18326, 50)
First few normalized gene symbols: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Normalized gene data saved to ../../output/preprocess/Stomach_Cancer/gene_data/GSE147163.csv
Created linked data with dummy trait column. Shape: (50, 18327)


Linked data shape after handling missing values: (50, 18327)
Trait distribution assessment: dataset is biased (all samples are gastric cancer)
Data quality check result: Not usable
Data not saved due to quality issues (no trait differentiation available).
